In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-email/spam.csv


In [2]:
data=pd.read_csv('/kaggle/input/spam-email/spam.csv')
print(data.shape) 
#in số dòng số cột trong data frame
data.head()
#in 5 dòng dữ liệu đầu

(5572, 2)


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
import re

In [4]:
# Hàm tiền xử lý dữ liệu
def preprocess_text(text):
    # Chuẩn hóa chữ hoa thành chữ thường
    text = text.lower()
    # Loại bỏ ký tự đặc biệt sử dụng biểu thức chính quy (regex)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text
#Biểu thức chính quy [^a-zA-Z0-9\s] áp dụng negation(phủ định) ('^') cho các ký tự không thuộc bảng chữ cái, 
#không phải chữ số và không phải khoảng trắng, và sau đó nó được thay thế bằng chuỗi rỗng (''), 
#có tác dụng loại bỏ chúng khỏi văn bản.

In [5]:
X = data['Message']
#Biến X chứa dữ liệu đầu vào là cột message
Y = data['Category']
#Biến Y chứa dữ liệu đầu vào là cột category

In [6]:
print(X)
print(Y)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object
0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: Category, Length: 5572, dtype: object


In [7]:
#Trước tiền xử lý DL 
for i in range(5):
    print(X.iloc[i])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Ok lar... Joking wif u oni...
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
U dun say so early hor... U c already then say...
Nah I don't think he goes to usf, he lives around here though


In [8]:
X = X.apply(preprocess_text)

In [9]:
#Sau tiền xl dl
for i in range(5):
    print(X.iloc[i])

go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat
ok lar joking wif u oni
free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry questionstd txt ratetcs apply 08452810075over18s
u dun say so early hor u c already then say
nah i dont think he goes to usf he lives around here though


In [10]:
from sklearn.model_selection import train_test_split
#Đoạn code này sử dụng hàm train_test_split từ thư viện sklearn.model_selection để chia dữ liệu 
#thành hai phần: một phần để huấn luyện mô hình (training set) và một phần để kiểm thử mô hình (test set).

In [11]:
# Phân chia dữ liệu
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=5)
#X_train chứa message train, X_test chứa message test, Y_train chứa category train, Y_test chứa ct test
#test_size: dữ liệu test chiếm 20%, còn lại 80% là train
#random_state=5: đảm bảo rằng việc chia dữ liệu sẽ luôn tạo ra kết quả giống nhau mỗi khi chạy đoạn code này

In [12]:
print(X_train)

1658    sif we have one good partnership going we will...
1509    sounds like something that someone testing me ...
3266    44 7732584351 do you want a new nokia 3510i co...
5199             ugh my leg hurts musta overdid it on mon
3217    urgent we are trying to contact u todays draw ...
                              ...                        
3046    hello yeah ive just got out of the bath and ne...
1725                    there bold 2  ltgt  is that yours
4079                   gam gone after outstanding innings
2254                          lol enjoy role playing much
2915    sorry u can not unsubscribe yet the mob offer ...
Name: Message, Length: 4457, dtype: object


In [13]:
# sử dụng CountVectorizer từ thư viện sklearn.feature_extraction.text 
#để chuyển đổi dữ liệu văn bản thành dạng ma trận đếm.
#CountVectorizer: Là một phương pháp biểu diễn văn bản thành vectơ dựa trên tần suất xuất hiện của các từ. 
#Nó đếm số lần xuất hiện của từng từ trong từng văn bản và tạo ra một ma trận, 
#trong đó mỗi hàng đại diện cho một văn bản và mỗi cột đại diện cho một từ trong tập từ vựng 
#của toàn bộ tập dữ liệu.
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
vect = CountVectorizer()
X_train = vect.fit_transform(X_train)
#Dòng này fit CountVectorizer với dữ liệu huấn luyện (X_train) và sau đó chuyển đổi dữ liệu huấn luyện 
#thành ma trận đếm. Quá trình "fit" sẽ xây dựng từ điển của các từ 
#từ dữ liệu huấn luyện và sau đó sử dụng từ điển này để chuyển đổi các văn bản thành ma trận đếm.
X_test  = vect.transform(X_test)
#Dòng này sử dụng từ điển đã học từ dữ liệu huấn luyện để chuyển đổi dữ liệu kiểm thử (X_test) thành ma trận đếm. 
#Điều này đảm bảo rằng cả dữ liệu huấn luyện và kiểm thử đều sử dụng cùng một bộ từ điển để biểu diễn.
#Sau khi thực hiện hai dòng này, X_train và X_test sẽ là các ma trận đếm tần suất xuất hiện của các từ 
#trong văn bản tương ứng của tập huấn luyện và tập kiểm thử.

In [15]:
print("Ma trận X_train sau khi áp dụng CountVectorizer:")
print(X_train)
print("\nMa trận X_test sau khi áp dụng CountVectorizer:")
print(X_test)

Ma trận X_train sau khi áp dụng CountVectorizer:
  (0, 6594)	1
  (0, 7920)	2
  (0, 3557)	1
  (0, 5323)	1
  (0, 3367)	1
  (0, 5488)	1
  (0, 3354)	1
  (0, 8041)	1
  (0, 7160)	1
  (0, 4353)	1
  (1, 6794)	1
  (1, 4416)	1
  (1, 6757)	1
  (1, 7285)	1
  (1, 6752)	1
  (1, 7253)	1
  (1, 4721)	1
  (1, 8144)	1
  (1, 6356)	1
  (2, 545)	1
  (2, 690)	1
  (2, 2543)	1
  (2, 8301)	1
  (2, 7885)	1
  (2, 5098)	1
  :	:
  (4456, 6120)	1
  (4456, 7294)	1
  (4456, 5269)	1
  (4456, 1763)	1
  (4456, 5192)	1
  (4456, 3544)	1
  (4456, 934)	1
  (4456, 3605)	1
  (4456, 8288)	1
  (4456, 4876)	1
  (4456, 5276)	1
  (4456, 4924)	1
  (4456, 6782)	1
  (4456, 4819)	1
  (4456, 7948)	1
  (4456, 6127)	1
  (4456, 3925)	1
  (4456, 5657)	1
  (4456, 7675)	1
  (4456, 7243)	1
  (4456, 610)	1
  (4456, 7306)	1
  (4456, 5433)	1
  (4456, 6162)	1
  (4456, 2893)	1

Ma trận X_test sau khi áp dụng CountVectorizer:
  (0, 26)	1
  (0, 152)	1
  (0, 383)	1
  (0, 556)	1
  (0, 692)	1
  (0, 863)	1
  (0, 1526)	1
  (0, 1725)	1
  (0, 1969)	1
  (0, 

In [16]:
#sử dụng mô hình Naive Bayes đa thức (MultinomialNB) từ thư viện sklearn.naive_bayes 
#để huấn luyện mô hình phân loại
from sklearn.naive_bayes import MultinomialNB

In [17]:
multNB = MultinomialNB()
#MultinomialNB(): Là một mô hình Naive Bayes được sử dụng cho dữ liệu có biểu diễn đa thức, 
#chẳng hạn như trong bài toán phân loại văn bản, khi chúng ta làm việc với các ma trận đếm từ vựng.
multNB.fit(X_train,Y_train)
# Dòng này thực hiện quá trình huấn luyện mô hình. 
#Các tham số đầu vào là X_train (ma trận đếm tần suất xuất hiện của các từ trong văn bản) 
#và Y_train (nhãn tương ứng với mỗi văn bản trong tập huấn luyện). 
#Mô hình sẽ học cách dự đoán nhãn của các văn bản dựa trên tần suất xuất hiện của các từ.
#Sau khi đoạn code này được thực hiện, multNB sẽ là một mô hình đã được huấn luyện 
#và có thể được sử dụng để dự đoán nhãn của các văn bản mới, chẳng hạn như dữ liệu trong X_test.

MultinomialNB()

In [18]:
Y_pred = multNB.predict(X_test)
#Dòng code này sử dụng mô hình Naive Bayes đã được huấn luyện (multNB) để dự đoán nhãn của dữ liệu kiểm thử (X_test).
#Đây là phương thức của mô hình Naive Bayes (MultinomialNB) trong thư viện scikit-learn (sklearn). 
#Phương thức này nhận đầu vào là dữ liệu kiểm thử (X_test), tức là ma trận đặc trưng của các văn bản 
#mà mô hình chưa nhìn thấy khi huấn luyện.

#Khi được gọi, phương thức này trả về một mảng các dự đoán cho nhãn tương ứng với từng mẫu trong X_test. 
#Điều này có nghĩa là Y_pred (biến được gán cho kết quả của phương thức) chứa các dự đoán của mô hình 
#với đầu vào là dữ liệu kiểm thử.

In [19]:
print("Ma trận dự đoán (Y_pred):")
print(Y_pred)

Ma trận dự đoán (Y_pred):
['spam' 'ham' 'spam' ... 'ham' 'spam' 'ham']


In [20]:
from sklearn.metrics import accuracy_score
#Hàm accuracy_score được sử dụng để tính toán độ chính xác của mô hình.

In [21]:
print(f'The accuracy score of this model is {accuracy_score(Y_test,Y_pred)}')
# In ra màn hình độ chính xác của mô hình. Độ chính xác được tính bằng cách so sánh 
#giữa nhãn thực tế (Y_test) và nhãn dự đoán (Y_pred). Hàm accuracy_score trả về tỷ lệ phần trăm 
#của các dự đoán chính xác trên tổng số mẫu.

The accuracy score of this model is 0.9883408071748879


In [22]:
# Nhập một email mẫu để kiểm tra
sample_email = "euuwiw find a words ....,niwoe!"

# Tiền xử lý email mẫu
sample_email = preprocess_text(sample_email)
sample_email_vectorized = vect.transform([sample_email])

# Dự đoán xem email có phải là spam hay không
prediction = multNB.predict(sample_email_vectorized)

# In kết quả
if prediction[0] == 'spam':
    print("Email này được dự đoán là spam.")
else:
    print("Email này được dự đoán là ham.")


Email này được dự đoán là ham.
